### Pré-processamento

In [1]:
# Importando libs para análise
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

%matplotlib inline

In [2]:
# Obtendo dados
df = pd.read_csv('dataset.csv')
df.columns = df.columns.str.lower()
df.head()

,title,content
0,"Ato Declaratorio Interpretativo SRF nº 15, de ...","Ato Declaratório Interpretativo SRF nº 15, de ..."
1,"Ato Declaratorio nº 01, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
2,"Ato Declaratorio nº 02, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
3,"Ato Declaratório nº 01, de 1998",ATO DECLARATÓRIO Nº 01/1998 07/01/1998 * Publi...
4,"Ato Declaratório nº 01, de 1999",ATO DECLARATÓRIO Nº 01/1999 14/01/1999 * Publi...


In [3]:
# Tamanho do dataset
df.shape[0]

3606

### Verificando por dados faltantes

In [4]:
# Verificando por dados faltantes
df.isnull().sum()

title      0
content    2
dtype: int64

In [5]:
# Removendo dados faltantes
df.dropna(inplace=True)
# Novo tamanho do dataset
df.shape[0]

3604

### Verificando por ambiguidades

In [6]:
tipos_de_documentos = ["ato declaratório", "ato declaratorio", "decreto", "instrução normativa", 
                      "lei", "lei complementar", "norma de execução", "nota explicativa"]

In [7]:
df['title'].head()

0    Ato Declaratorio Interpretativo SRF nº 15, de ...
1                      Ato Declaratorio nº 01, de 2013
2                      Ato Declaratorio nº 02, de 2013
3                      Ato Declaratório nº 01, de 1998
4                      Ato Declaratório nº 01, de 1999
Name: title, dtype: object

In [8]:
def num_ocorrencias(documento):
    """Conta o número de ocorrências para cada tipo de classe de documento"""
    return df['title'].apply(lambda t: bool(re.match(documento, t.lower()))).sum()

def indices_ocorrencias(documento):
    """Gera uma lista de índices para cada ocorrência de tipo de classe de documento"""
    return df.index[df['title'].apply(lambda t: bool(re.match(documento, t.lower())))].tolist()


dist_documentos = [num_ocorrencias(documento) for documento in tipos_de_documentos]
indices_documentos = [indices_ocorrencias(documento) for documento in tipos_de_documentos]

In [9]:
indice = 0

print("Distribuição das classes:\n")

for documento, qtd in zip(tipos_de_documentos, dist_documentos):
    print(indice, documento, qtd)
    indice += 1

print("\nTotal:", sum(dist_documentos))

Distribuição das classes:

0 ato declaratório 669
1 ato declaratorio 3
2 decreto 1024
3 instrução normativa 1376
4 lei 331
5 lei complementar 72
6 norma de execução 117
7 nota explicativa 84

Total: 3676


Aparentemente, as classes 0 e 1 pertencem ao mesmo grupo (*ato declaratório*). Vamos defini-las como uma só:

In [10]:
# Índices das tuplas com título "ato declaratorio"
print(indices_documentos[1])

for i in range(3):
    df['title'].iloc[i] = df['title'].iloc[i].replace('Declaratorio', 'Declaratório')
    
df['title'].iloc[:3]

[0, 1, 2]


0    Ato Declaratório Interpretativo SRF nº 15, de ...
1                      Ato Declaratório nº 01, de 2013
2                      Ato Declaratório nº 02, de 2013
Name: title, dtype: object

Há alguma diferença no número de exemplares de classes total e o número total de dados?

In [11]:
df.shape[0], sum(dist_documentos)

(3604, 3676)

In [12]:
conjs_indices_documentos = [set(lista) for lista in indices_documentos]

# Observando se há interseções entre os índices dois a dois (i.e. se há dados com duas classes)
for i in range(0, 8):
    for j in range(0, 8):
        if i != j and set.intersection(conjs_indices_documentos[i], conjs_indices_documentos[j]):
            print(f"i: {i} j: {j}")

i: 4 j: 5
i: 5 j: 4


In [13]:
intersect_lista = [el for el in set.intersection(conjs_indices_documentos[4], conjs_indices_documentos[5])]
df['title'].iloc[df.index[intersect_lista]]

3077    Lei Complementar 0024, de 1975 (Cópia em Traba...
3078    Lei Complementar 0024, de 1975 (Cópia em Traba...
3079    Lei Complementar 0024, de 1975 (Cópia em Traba...
3080    Lei Complementar 0024, de 1975 (Cópia em Traba...
3081    Lei Complementar 0024, de 1975 (Cópia em Traba...
                              ...                        
3144                      Lei Complementar nº 50, de 2004
3145                      Lei Complementar nº 81, de 2009
3146                      Lei Complementar nº 81, de 2009
3147                               Lei n° 08.137, de 1990
3148                               Lei n° 08.397, de 1992
Name: title, Length: 72, dtype: object

Os dados supostamente ambíguos referem-se a títulos com a palavra "lei", mas definimos duas classes diferentes, i.e., "lei" e "lei complementar". Uma vez que a palavra *lei* é encontrada em ambos, não há problemas. Como a classe é contada duas vezes, retiremos a diferença:

In [14]:
sum(dist_documentos) - len(intersect_lista), df.shape[0] # (Diferença, Tamanho real do dataset)

(3604, 3604)

Aparentemente, não há ambiguidade nos títulos. Podemos separar os dados nas classes que definimos.

In [15]:
df.head()

,title,content
0,"Ato Declaratório Interpretativo SRF nº 15, de ...","Ato Declaratório Interpretativo SRF nº 15, de ..."
1,"Ato Declaratório nº 01, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
2,"Ato Declaratório nº 02, de 2013",* Publicado no DOE em 01/02/2013 ATO DECLARA...
3,"Ato Declaratório nº 01, de 1998",ATO DECLARATÓRIO Nº 01/1998 07/01/1998 * Publi...
4,"Ato Declaratório nº 01, de 1999",ATO DECLARATÓRIO Nº 01/1999 14/01/1999 * Publi...


### Separando o dataset em classes

In [16]:
classes = ["ato declaratório", "decreto", "instrução normativa", "lei", "norma de execução", "nota explicativa"]

In [17]:
dist_classes = [num_ocorrencias(classe) for classe in classes]
{classe: qtd for classe, qtd in zip(classes, dist_classes)}

{'ato declaratório': 672,
 'decreto': 1024,
 'instrução normativa': 1376,
 'lei': 331,
 'norma de execução': 117,
 'nota explicativa': 84}

In [18]:
def define_classe(titulo):
    """Retorna a classe adequada para o titulo fornecido"""
    for classe in classes:
        if re.match(classe, titulo.lower()):
            return classe

df['doc_type'] = df['title'].apply(define_classe)        

In [23]:
# Visualização da distribuição dos documentos
plt.figure(figsize=(12, 7))
sns.histplot(df['doc_type'])
plt.xlabel('Tipo de documento')
plt.ylabel('Quantidade')
plt.show()

AttributeError: module 'seaborn' has no attribute 'histplot'

<Figure size 864x504 with 0 Axes>

In [25]:
# Valores absolutos
df['doc_type'].value_counts()

instrução normativa    1376
decreto                1024
ato declaratório        672
lei                     331
norma de execução       117
nota explicativa         84
Name: doc_type, dtype: int64

## Classificação

Import das bibliotecas necessárias

In [39]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

Separação dos dados em treino e teste

In [27]:
X = df['content']
y = df['doc_type']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Vetorização dos dados usando CountVectorizer e TFidVectorizer

In [31]:
c_vec = CountVectorizer()
t_vec = TfidfVectorizer()

X_train_c = c_vec.fit_transform(X_train)
X_train_t = t_vec.fit_transform(X_train)

X_test_c = c_vec.transform(X_test)
X_test_t = t_vec.transform(X_test)

### Teste com classificadores: NaiveBayes, Decision Tree e Random Forest

NaiveBayes com CountVectorizer

In [35]:
clf = MultinomialNB().fit(X_train_c, y_train)

pred = clf.predict(X_test_c)

print(classification_report(y_test, pred))

cm = confusion_matrix(y_test, pred)
print(cm)

                     precision    recall  f1-score   support

   ato declaratório       1.00      0.96      0.98       243
            decreto       0.49      0.98      0.65       340
instrução normativa       0.99      0.39      0.56       438
                lei       0.86      0.73      0.79       104
  norma de execução       1.00      0.29      0.45        38
   nota explicativa       0.00      0.00      0.00        27

           accuracy                           0.69      1190
          macro avg       0.72      0.56      0.57      1190
       weighted avg       0.81      0.69      0.67      1190

[[234   7   2   0   0   0]
 [  0 333   0   7   0   0]
 [  1 265 170   2   0   0]
 [  0  28   0  76   0   0]
 [  0  26   0   1  11   0]
 [  0  25   0   2   0   0]]


C:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Naivy Bayes com TFidVectorizer

In [34]:
clf = MultinomialNB().fit(X_train_t, y_train)

pred = clf.predict(X_test_t)

print(classification_report(y_test, pred))

cm = confusion_matrix(y_test, pred)
print(cm)

                     precision    recall  f1-score   support

   ato declaratório       1.00      0.69      0.81       243
            decreto       0.76      0.93      0.83       340
instrução normativa       0.72      0.99      0.83       438
                lei       0.00      0.00      0.00       104
  norma de execução       0.00      0.00      0.00        38
   nota explicativa       0.00      0.00      0.00        27

           accuracy                           0.77      1190
          macro avg       0.41      0.43      0.41      1190
       weighted avg       0.68      0.77      0.71      1190

[[167   0  76   0   0   0]
 [  0 316  24   0   0   0]
 [  0   4 434   0   0   0]
 [  0  91  13   0   0   0]
 [  0   2  36   0   0   0]
 [  0   5  22   0   0   0]]


C:\Users\lucas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Decision Tree com CountVectorizer

In [41]:
clf = tree.DecisionTreeClassifier().fit(X_train_c, y_train)

pred = clf.predict(X_test_c)

print(classification_report(y_test, pred))

cm = confusion_matrix(y_test, pred)
print(cm)

                     precision    recall  f1-score   support

   ato declaratório       0.99      1.00      1.00       243
            decreto       0.99      0.99      0.99       340
instrução normativa       0.99      1.00      0.99       438
                lei       0.99      0.99      0.99       104
  norma de execução       0.92      0.87      0.89        38
   nota explicativa       1.00      0.93      0.96        27

           accuracy                           0.99      1190
          macro avg       0.98      0.96      0.97      1190
       weighted avg       0.99      0.99      0.99      1190

[[243   0   0   0   0   0]
 [  0 336   2   0   2   0]
 [  1   0 436   0   1   0]
 [  0   1   0 103   0   0]
 [  0   2   2   1  33   0]
 [  1   0   1   0   0  25]]


Decision Tree com TFidVectorizer

In [37]:
clf = tree.DecisionTreeClassifier().fit(X_train_t, y_train)

pred = clf.predict(X_test_t)

print(classification_report(y_test, pred))

cm = confusion_matrix(y_test, pred)
print(cm)

                     precision    recall  f1-score   support

   ato declaratório       1.00      0.98      0.99       243
            decreto       1.00      1.00      1.00       340
instrução normativa       0.99      1.00      0.99       438
                lei       0.99      1.00      1.00       104
  norma de execução       0.95      0.92      0.93        38
   nota explicativa       0.89      0.93      0.91        27

           accuracy                           0.99      1190
          macro avg       0.97      0.97      0.97      1190
       weighted avg       0.99      0.99      0.99      1190

[[239   0   3   0   0   1]
 [  0 339   0   0   1   0]
 [  0   0 436   1   1   0]
 [  0   0   0 104   0   0]
 [  0   0   1   0  35   2]
 [  0   0   2   0   0  25]]


Random Forest com CountVectorizer

In [44]:
clf = RandomForestClassifier().fit(X_train_c, y_train)

pred = clf.predict(X_test_c)

print(classification_report(y_test, pred))

cm = confusion_matrix(y_test, pred)
print(cm)

                     precision    recall  f1-score   support

   ato declaratório       1.00      1.00      1.00       243
            decreto       0.99      1.00      0.99       340
instrução normativa       0.91      1.00      0.95       438
                lei       1.00      0.97      0.99       104
  norma de execução       1.00      0.32      0.48        38
   nota explicativa       0.88      0.26      0.40        27

           accuracy                           0.96      1190
          macro avg       0.96      0.76      0.80      1190
       weighted avg       0.96      0.96      0.95      1190

[[242   0   1   0   0   0]
 [  0 340   0   0   0   0]
 [  0   2 436   0   0   0]
 [  0   3   0 101   0   0]
 [  1   0  24   0  12   1]
 [  0   0  20   0   0   7]]


Random Forest com TFidVectorizer

In [40]:
clf = RandomForestClassifier().fit(X_train_t, y_train)

pred = clf.predict(X_test_t)

print(classification_report(y_test, pred))

cm = confusion_matrix(y_test, pred)
print(cm)

                     precision    recall  f1-score   support

   ato declaratório       1.00      1.00      1.00       243
            decreto       1.00      1.00      1.00       340
instrução normativa       0.90      1.00      0.95       438
                lei       1.00      0.99      1.00       104
  norma de execução       1.00      0.32      0.48        38
   nota explicativa       1.00      0.19      0.31        27

           accuracy                           0.96      1190
          macro avg       0.98      0.75      0.79      1190
       weighted avg       0.96      0.96      0.95      1190

[[242   0   1   0   0   0]
 [  0 340   0   0   0   0]
 [  0   0 438   0   0   0]
 [  0   1   0 103   0   0]
 [  1   0  25   0  12   0]
 [  0   0  22   0   0   5]]
